# GraphPlot tutorial

**Tutorial in Working progress**

If you have read the previous tutorials you can run this notebook and have fun playing with the widget. If not... play and see what happens!

This is a basic tutorial on how to use the shaolin GraphPlot for interactive visualization of graphs.

### Imports and setup of sample data

In [1]:
#wide screen hack, great for making big plots and have more room for the control panels
from IPython.core.display import HTML
html ='''<style>
.container { width:98% !important; }
.input{ width:40% !important; }
.text_cell{ width:40% !important;
            font-size: 16px;}
.title {align:center !important;}
</style>'''
HTML(html)

In [1]:
import sys
path ="/home/sergio/github/shaolin-master"
sys.path.append(path)

In [2]:
from IPython.display import Image #this is for displaying the widgets in the web version of the notebook
import pandas as pd
from shaolin.benders.graph import GraphPlot
from shaolin.wranglers.graph import GraphCalculator

Loading BokehJS ...

In [3]:
node_metrics = pd.read_hdf('gplot_data/sample_node_metrics.h5')
matrix_panel = pd.read_hdf('gplot_data/sample_matrix_panel.h5')

In [4]:
gc = GraphCalculator(node_metrics=node_metrics,matrix_panel=matrix_panel)

## Plot initialization

To start plotting a graph you only need to instantiate a GraphPlot holding the information we want to visualize. Al the managemen can be done as usual using the *widget* attribute.

**Alpha version disclaimer**:

- As the GraphCalculator still doesn't handle the *node_metrics* it is necessary to merge them with the *node* Dataframe by hand in order to handle the tooltip info propperly. This will soon be corrected.

In [5]:
gc.node_metrics = gc.node_metrics.T.combine_first(gc.node) #Alpha version hack

In [6]:
gp = GraphPlot(gc)

In [9]:
gp.widget